In [17]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd
import math

In [18]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A_3.csv")
pi = pd.read_csv("pi_3.csv")
pi = np.array(pi)
pi = pi.T
A = np.array(A)
b = np.array([1000 for _ in range(m)])  


In [19]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)
prob.solve()
opv1 = prob.value
print("offline optimal value, ", opv1)


offline optimal value,  56275.0128878739


In [20]:
Anow = A
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)
piNow = np.reshape(piNow, (n,))
bNow = b
bNow = bNow.T

temp = m+n
y = cp.Variable(temp)
e = np.ones(n)
objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cp.Problem(objective, constraints)

prob.solve()
P = y[:m].value
p_truth = P
p_truth

array([ 1.04581214,  2.05604972,  3.06239824,  4.07860401,  5.09664173,
        6.06381518,  7.06742707,  8.07094188,  9.0689332 , 10.08168534])

In [21]:
def u(s,w):
    value = 0
    for i in range(m):
        value += w/m * cp.log(s[i])
    return value
def learning(k=50,w=25):
    A1 = A[:m, :k]
    # pi = np.array(pi)
    pi1 = pi[:,:k]
    x = cp.Variable(k)
    s = cp.Variable(m)
    # print(pi1.shape)
    # print(x.shape)
    obj = cp.Maximize(pi1@x + u(s,w))
    cons = [A1@x + s <= k/n * b, x>=0, x<=1]
    prob = cp.Problem(obj, cons)
    # prob.solve(solver="ECOS")
    prob.solve(solver="SCS")
    opt_sol = x.value
    dual_price = prob.constraints[0].dual_value
    opt_value = prob.value 
    return opt_sol, dual_price, opt_value

def decision(dual_price,k=50):
    # Make the decisions based on the dual prices
    AT = A.T
    xp = []
    xk = []
    for i in range(n):
        cri = (AT[i].dot(dual_price))
        if(pi[0][i] > cri):
            xp.append(1)
        else:
            xp.append(0)  # xt(p) 10000
        if i < k:
            xk.append(0)  #50
    # Check whether the resource constriant is satisfied
    for t in range(k,n):
        flag = 1
        for j in range(m):
            temp1 = A[j,t]*xp[t]
            temp2 = b[j] - (A[j, :t]).dot(np.array(xk))
            if temp1 > temp2:
                flag = 0
                break
        if flag == 1:
            xk.append(xp[t])
        else:
            xk.append(0)
    return xk

**K = 50**

In [22]:
k = 50 
w = 17
opt_sol, dual_price, opt_value = learning(k,w)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]

ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  50
dual price is [1.42001053 1.98939983 2.83236566 4.23279761 5.14573515 5.96428342
 6.80344799 7.91890321 9.07183978 9.96573854]
objective value is  52639.585341734484
the competetive ratio is  0.9353989033572878


**K = 100**

In [23]:
k = 100
w = 17
opt_sol, dual_price, opt_value = learning(k,w)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  100
dual price is [1.28411034 1.90980371 2.77602463 4.27778174 5.0042637  6.00917886
 7.0020034  8.19085423 9.16960693 9.91471533]
objective value is  47908.214203690775
the competetive ratio is  0.8513230250012791


**K = 200**

In [24]:
k = 200
w = 25
opt_sol, dual_price, opt_value = learning(k,w)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  200
dual price is [ 1.22684355  2.04817876  2.83011104  4.21479058  5.1074395   6.09846749
  7.03956316  8.02802266  9.03101244 10.01374096]
objective value is  51357.924201157795
the competetive ratio is  0.9126239438360816


In [25]:


dic_ratio = {}
dic_price = {}
for k in range(50, 1000, 50):
    opt_sol, dual_price, opt_value = learning(k,17)
    l_decision = decision(dual_price, k)
    opv2 = (pi.dot(l_decision))[0]
    ratio = opv2/opv1
    dic_ratio[k] = ratio
    dic_price[k] = dual_price
    print("when k = ", k)
    print("competetive ratio is", ratio)
    print("norm of dual price difference", np.linalg.norm(dual_price-p_truth))

when k =  50
competetive ratio is 0.9353989033572878
norm of dual price difference 0.5828609251255498
when k =  100
competetive ratio is 0.8513230250012791
norm of dual price difference 0.5177333032418509
when k =  150
competetive ratio is 0.9025762441630031
norm of dual price difference 0.3725447237337063
when k =  200
competetive ratio is 0.9084901172615253
norm of dual price difference 0.3355597750076605
when k =  250
competetive ratio is 0.8990209575307745
norm of dual price difference 0.3085764418168975
when k =  300
competetive ratio is 0.9525137747842355
norm of dual price difference 0.15140187532392185
when k =  350
competetive ratio is 0.926849275826751
norm of dual price difference 0.1853765671021461
when k =  400
competetive ratio is 0.9475003540161019
norm of dual price difference 0.14731021100985842
when k =  450
competetive ratio is 0.9285571839929508
norm of dual price difference 0.15126869844124177
when k =  500
competetive ratio is 0.9134476637322889
norm of dual price